In [6]:
%pylab inline
import pandas as pd
import h5py
from tqdm import tqdm


Populating the interactive namespace from numpy and matplotlib


In [2]:
store = pd.HDFStore('store.h5')

In [3]:
df= pd.read_table('/staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679/ribocop_results_Feb2019_longest/SRX118287_translating_ORFs.tsv')

In [5]:
df.head()

,ORF_ID,ORF_type,status,phase_score,read_count,length,valid_codons,transcript_id,transcript_type,gene_id,gene_name,gene_type,chrom,strand,start_codon,profile
0,ENST00000335137.3_69091_70005_915,annotated,nontranslating,0.000000,0,915,0,ENST00000335137.3,protein_coding,ENSG00000186092.4,OR4F5,protein_coding,chr1,+,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,ENST00000624431.1_182709_184155_399,annotated,nontranslating,0.000000,0,399,0,ENST00000624431.1,protein_coding,ENSG00000279928.1,FO538757.2,protein_coding,chr1,+,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,ENST00000623834.3_185220_195411_1077,annotated,nontranslating,1.000000,2,1077,1,ENST00000623834.3,protein_coding,ENSG00000279457.3,FO538757.1,protein_coding,chr1,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,ENST00000623083.3_185220_195411_1392,annotated,nontranslating,1.000000,2,1392,1,ENST00000623083.3,protein_coding,ENSG00000279457.3,FO538757.1,protein_coding,chr1,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,ENST00000624735.1_184930_200086_1365,annotated,nontranslating,0.333333,7,1365,3,ENST00000624735.1,protein_coding,ENSG00000279457.3,FO538757.1,protein_coding,chr1,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
h5py_obj.close()

In [13]:
row

ORF_ID                             ENST00000335137.3_69091_70005_915
ORF_type                                                   annotated
status                                                nontranslating
phase_score                                                        0
read_count                                                         0
length                                                           915
valid_codons                                                       0
transcript_id                                      ENST00000335137.3
transcript_type                                       protein_coding
gene_id                                            ENSG00000186092.4
gene_name                                                      OR4F5
gene_type                                             protein_coding
chrom                                                           chr1
strand                                                             +
start_codon                       

In [14]:
import h5py
h5py_obj = h5py.File ('/staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679/ribocop_results_Feb2019_longest/SRX118287_translating_ORFs.h5', 'w')
total_rows = df.shape[0]
with tqdm(total=total_rows) as pbar:
    for idx, row in df.iterrows():
        orf_id = row['ORF_ID']
        profile = eval(row['profile'])
        read_count = row['read_count']
        phase_score = row['phase_score']
        valid_codons = row['valid_codons']
        subgroup = h5py_obj.create_group(orf_id)
        
        dset = subgroup.create_dataset(
            "profile",
            (len(profile),),
            dtype=np.dtype("int64"),
            compression="gzip",
            compression_opts=9,
        )
        dset[...] = profile

        dset = subgroup.create_dataset(
            "read_count",
            (1,),
            dtype=np.dtype("int64"),
            compression="gzip",
            compression_opts=9,
        )
        dset[...] = [read_count]
        
        dset = subgroup.create_dataset(
            "valid_codons",
            (1,),
            dtype=np.dtype("int64"),
            compression="gzip",
            compression_opts=9,
        )
        dset[...] = [valid_codons]

        dset = subgroup.create_dataset(
            "phase_score",
            (1,),
            dtype=np.dtype("int64"),
            compression="gzip",
            compression_opts=9,
        )
        dset[...] = [phase_score]
        pbar.update()

 27%|██▋       | 638595/2397475 [2:43:07<34:42:21, 14.08it/s]


KeyboardInterrupt: 

In [ ]:
class HDFParser(object):
    def __init__(self, filepath):
        self.filepath = filepath
        self.h5py_obj = h5py.File(filepath, "r")
        chrom_names = self.h5py_obj["chrom_names"]
        chrom_sizes = self.h5py_obj["chrom_sizes"]
        chrom_lengths = dict(
            [
                (chrom.replace("_neg", "").replace("_pos", ""), int(size))
                for chrom, size in zip(chrom_names, chrom_sizes)
            ]
        )
        chrom_lengths = OrderedDict(sorted(chrom_lengths.items()))
        self.chromosome_lengths = chrom_lengths

    def close(self):
        self.h5py_obj.close()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.h5py_obj.close()

    def get_coverage(
        self,
        region=(None, None, None, None),
        fragment_length=None,
        orientation="5prime",
        outprefix=None,
    ):
        """Get coverage for a selected region.

        Since the region is user provided, we assume the user is aware if it is strand
        specfic or not

        Paramters
        ---------
        region: tuple
                (chromosome, start[0-based], end[1-based], strand(+/-))
        fragment_length: list or string
                         list of fragment lengths to use
                         Example: [20, 21, 22] or 'all'

        """
        chrom, start, stop, strand = region
        start = int(start)
        stop = int(stop)
        assert chrom is not None, "chromosome not set"
        assert start is not None, "start position is not set"
        assert stop is not None, "end position not set"
        assert strand is not None, "strand is not set"
        assert start < stop, "start should be < stop"
        assert fragment_length is not None, "fragment_length not set"
        if isinstance(fragment_length, int):
            fragment_length = [fragment_length]

        if fragment_length == "all":
            h5py_fragments = self.h5py_obj["read_lengths"]
            fragment_length = h5py_fragments
        fragment_length = list([str(x) for x in fragment_length])
        coverages_normalized = pd.DataFrame()
        coverages = pd.DataFrame()
        position_counter = Counter()
        for l in fragment_length:
            if strand == "-":
                chrom_name = "{}__neg".format(chrom)
            elif strand == "+":
                chrom_name = "{}__pos".format(chrom)
            else:
                raise ValueError("strand ill-defined: {}".format(strand))
            root_obj = self.h5py_obj["fragments"][l][orientation]
            if chrom_name not in list(root_obj.keys()):
                # This chromosome does not exist in the
                # key value store
                # So should returns zeros all way
                coverage = pd.Series(
                    [0] * (stop - start), index=list(range(start, stop))
                )

            else:
                chrom_obj = root_obj[chrom_name]
                counts_series = pd.Series(
                    list(chrom_obj["counts"]), index=list(chrom_obj["positions"])
                )
                # print(counts_series)
                coverage = counts_series.get(list(range(start, stop)))
                if coverage is None:
                    coverage = pd.Series(
                        [0] * (stop - start), index=list(range(start, stop))
                    )
                coverage = coverage.fillna(0)
            # Mean is taken by summing the rows
            coverage_mean = coverage.mean(axis=0, skipna=True)
            # to normalize
            # we divide the sum by vector obtained in previous
            # such that each column gets divided
            coverage_normalized = coverage.divide(coverage_mean).fillna(0)
            coverages = coverages.join(
                pd.DataFrame(coverage, columns=[str(l)]), how="outer"
            )
            coverages_normalized = coverages_normalized.join(
                pd.DataFrame(coverage_normalized, columns=[str(l)]), how="outer"
            )
            position_counter += Counter(coverage.index.tolist())
        position_counter = pd.Series(Counter(position_counter)).sort_index()
        coverage_sum = coverages.sum(axis=1)
        coverage_normalized_sum = coverages_normalized.sum(axis=1)
        coverage_sum_div = coverage_sum.divide(position_counter, axis="index")
        coverage_normalized_sum_div = coverage_normalized_sum.divide(
            position_counter, axis="index"
        )
        if outprefix:
            mkdir_p(os.path.dirname(outprefix))
            coverage_sum_div.to_csv("{}_raw.tsv", index=False, header=True, sep="\t")
            coverage_normalized_sum_div.to_csv(
                "{}_normalized.tsv", index=False, header=True, sep="\t"
            )

        coverages.index.name = "start"
        coverages = coverages.reset_index()
        coverages_normalized.index.name = "start"
        coverages_normalized = coverages_normalized.reset_index()
        return (
            coverages,
            coverages_normalized,
            coverage_sum_div,
            coverage_normalized_sum_div,
        )

    def get_read_length_dist(self):
        read_lengths = list([int(x) for x in self.h5py_obj["read_lengths"]])
        read_counts = list(self.h5py_obj["read_lengths_counts"])
        return pd.Series(read_counts, index=read_lengths).sort_index()

    def get_query_alignment_length_dist(self):
        read_lengths = list([int(x) for x in self.h5py_obj["query_alignment_lengths"]])
        read_counts = list(self.h5py_obj["query_alignment_lengths_counts"])
        return pd.Series(read_counts, index=read_lengths).sort_index()